In [1]:
import pandas as pd
import numpy as np

import joblib
from flask import Flask, render_template, request
import shap

app = Flask(__name__)

# Load the pickled model

model = joblib.load('model_randomforest_treatment6.pkl')
explainer = shap.TreeExplainer(model)

In [2]:
questions = {
        1  : "If one of us apologizes when our discussion deteriorates, the discussion ends.",
        2  : "I know we can ignore our differences, even if things get hard sometimes.",
        3  : "When we need it, we can take our discussions with my spouse from the beginning and correct it.",
        4  : "When I discuss with my spouse, to contact him will eventually work.",
        5  : "The time I spent with my wife is special for us.",
        6  : "We don't have time at home as partners.",
        7  : "We are like two strangers who share the same environment at home rather than family.",
        8  : "I enjoy our holidays with my wife.",
        9  : "I enjoy traveling with my wife.",
        10 : "Most of our goals are common to my spouse.",
        11 : "I think that one day in the future, when I look back, I see that my spouse and I have been in harmony with each other.",
        12 : "My spouse and I have similar values in terms of personal freedom.",
        13 : "My spouse and I have similar sense of entertainment.",
        14 : "Most of our goals for people (children, friends, etc.) are the same.",
        15 : "Our dreams with my spouse are similar and harmonious.",
        16 : "We're compatible with my spouse about what love should be.",
        17 : "We share the same views about being happy in our life with my spouse",
        18 : "My spouse and I have similar ideas about how marriage should be",
        19 : "My spouse and I have similar ideas about how roles should be in marriage",
        20 : "My spouse and I have similar values in trust.",
        21 : "I know exactly what my wife likes.",
        22 : "I know how my spouse wants to be taken care of when she/he sick.",
        23 : "I know my spouse's favorite food.",
        24 : "I can tell you what kind of stress my spouse is facing in her/his life.",
        25 : "I have knowledge of my spouse's inner world.",
        26 : "I know my spouse's basic anxieties.",
        27 : "I know what my spouse's current sources of stress are.",
        28 : "I know my spouse's hopes and wishes.",
        29 : "I know my spouse very well.",
        30 : "I know my spouse's friends and their social relationships.",
        31 : "I feel aggressive when I argue with my spouse.",
        32 : "When discussing with my spouse, I usually use expressions such as ‘you always’ or ‘you never’ .",
        33 : "I can use negative statements about my spouse's personality during our discussions.",
        34 : "I can use offensive expressions during our discussions.",
        35 : "I can insult my spouse during our discussions.",
        36 : "I can be humiliating when we discussions.",
        37 : "My discussion with my spouse is not calm.",
        38 : "I hate my spouse's way of open a subject.",
        39 : "Our discussions often occur suddenly.",
        40 : "We're just starting a discussion before I know what's going on.",
        41 : "When I talk to my spouse about something, my calm suddenly breaks.",
        42 : "When I argue with my spouse, ı only go out and I don't say a word.",
        43 : "I mostly stay silent to calm the environment a little bit.",
        44 : "Sometimes I think it's good for me to leave home for a while.",
        45 : "I'd rather stay silent than discuss with my spouse.",
        46 : "Even if I'm right in the discussion, I stay silent to hurt my spouse.",
        47 : "When I discuss with my spouse, I stay silent because I am afraid of not being able to control my anger.",
        48 : "I feel right in our discussions.",
        49 : "I have nothing to do with what I've been accused of.",
        50 : "I'm not actually the one who's guilty about what I'm accused of.",
        51 : "I'm not the one who's wrong about problems at home.",
        52 : "I wouldn't hesitate to tell my spouse about her/his inadequacy.",
        53 : "When I discuss, I remind my spouse of her/his inadequacy.",
        54 : "I'm not afraid to tell my spouse about her/his incompetence."
}

@app.route('/', methods=['GET'])
def index():
    return render_template('form.html', questions=questions)

@app.route('/predict', methods=['POST'])
def predict():
    responses = []
    for qid in questions:
        response = request.form.get(f'q{qid}')  # Get the user's response for the question
        if response is not None and response != '':
            responses.append(int(response))
        else:
            responses.append(0)  # Use 0 as the default value for missing responses

    # Convert responses to a format suitable for prediction
    X = [responses]

    # Predict divorce probability using the loaded model
    prediction =  model.predict_proba(X)[0][0] * 100 # Assuming the model provides the probability of class 1 directly

    shap_values = explainer.shap_values(pd.DataFrame(X, columns=questions.keys()))

    abs_shap_values = np.abs(shap_values[1][0])

    top_feature_indices = np.argsort(abs_shap_values)[::-1][:5]

    # Retrieve the top feature names from the questions dictionary
    top_feature_names = [questions[qid] for qid in top_feature_indices]

    top_feature_values = [responses[i] for i in top_feature_indices]
    top_feature_shap_values = shap_values[1][0][top_feature_indices]

    return render_template('result.html', prediction=prediction, top_feature_names=top_feature_names, top_feature_values=top_feature_values, top_feature_shap_values=top_feature_shap_values)

In [3]:
if __name__ == '__main__':
    app.run()    
#     app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Aug/2023 18:33:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2023 18:33:54] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [14/Aug/2023 18:33:54] "GET /static/desktop-wallpaper-divorce.jpg HTTP/1.1" 304 -
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
127.0.0.1 - - [14/Aug/2023 18:33:59] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2023 18:33:59] "GET /static/style.css HTTP/1.1" 304 -
